In [1]:
import sys
print("Python version:", sys.version)
print("Python interpreter path:", sys.executable)

Python version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:27:36) [GCC 11.2.0]
Python interpreter path: /vol/home/s4422090/miniconda3/envs/IDL/bin/python


In [2]:
!nvidia-smi

Sat Nov  2 21:23:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060        Off | 00000000:01:00.0 Off |                  N/A |
| 32%   31C    P8              N/A / 115W |     69MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2024-11-02 21:23:34.338969: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-02 21:23:34.348179: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730579014.356640 1071258 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730579014.359180 1071258 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 21:23:34.368777: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Num GPUs Available:  1


In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPUs")
    except RuntimeError as e:
        print(e)

Memory growth set for GPUs


In [ ]:
'''
Trains a simple convnet on the MNIST dataset using TensorFlow with GPU support.

Achieves around 99.25% test accuracy after 12 epochs.
'''


import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import os



# Verify TensorFlow installation and GPU availability
print("TensorFlow version:", tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
print("Num GPUs Available:", len(gpus))

if gpus:
    try:
        # Enable memory growth for each GPU to prevent TensorFlow from allocating all GPU memory at once
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print("Error setting memory growth:", e)
else:
    print("No GPU detected. Using CPU.")

# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 12

# Input image dimensions
img_rows, img_cols = 28, 28

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices (one-hot encoding)
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Display model summary
model.summary()

# Train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

TensorFlow version: 2.18.0
Num GPUs Available: 1
1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1730579042.563675 1071258 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6231 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


/vol/home/s4422090/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

...

Epoch 1/12


I0000 00:00:1730579049.016658 1071414 service.cc:148] XLA service 0x7fb6a400a590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730579049.017180 1071414 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2024-11-02 21:24:09.451200: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1730579050.106720 1071414 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-02 21:24:10.380642: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
I0000 00:00:173057

469/469 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.1182 - loss: 2.2906 - val_accuracy: 0.3325 - val_loss: 2.2438
Epoch 2/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2345 - loss: 2.2361 - val_accuracy: 0.5768 - val_loss: 2.1693
Epoch 3/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3631 - loss: 2.1618 - val_accuracy: 0.7072 - val_loss: 2.0635
Epoch 4/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4588 - loss: 2.0581 - val_accuracy: 0.7594 - val_loss: 1.9176
Epoch 5/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5340 - loss: 1.9190 - val_accuracy: 0.7857 - val_loss: 1.7295
Epoch 6/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5760 - loss: 1.7449 - val_accuracy: 0.7956 - val_loss: 1.5121
Epoch 7/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6056 - loss: 1.5607 - val_accuracy: 0.8090 - val_loss: 1.2943
Epoch 8/12
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6389 - loss: 1.3806 - val_accuracy: 0.8201 - val

2024-11-02 21:24:38.308377: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_72', 4 bytes spill stores, 4 bytes spill loads



Test loss: 0.6866962909698486
Test accuracy: 0.8529999852180481


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split

def load_and_split_data(images_path, labels_path, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, random_state=42):


    images = np.load(images_path)
    labels = np.load(labels_path)

    # Normalize the images to [0,1]
    images = images.astype('float32') / 255.0 * np.pi
    images = np.sin(images)

    num_samples, height, width = images.shape


    # Suffle + separete into training, validation, test
    X_train, X_temp, y_train, y_temp = train_test_split(
        images, labels, 
        test_size= (1 - train_ratio), 
        random_state=random_state, 
        shuffle=True
    )

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, 
        test_size= val_ratio / (test_ratio + val_ratio), 
        random_state=random_state, 
        shuffle=True
    )   
    """
    print(images.shape)
    print(f"Training set shape: {X_train.shape}, {y_train.shape}")
    print(f"Validation set shape: {X_val.shape}, {y_val.shape}")
    print(f"Test set shape: {X_test.shape}, {y_test.shape}")
    """
    
    return [X_train, y_train, X_val, y_val, X_test, y_test]

def label_transformation(labels):
    
    hours = labels[:, 0]
    minutes = labels[:, 1]
    
    # Base label
    new_labels = 2*hours

    # Increment label by 1 if minute > 30
    new_labels += (minutes > 30).astype(int)

    return new_labels


if __name__ == "__main__":

    images_path = r'C:\Users\baroi\OneDrive\Desktop\Leiden University CS - AI\Deep Learning\Assignment 1\images.npy'
    labels_path = r'C:\Users\baroi\OneDrive\Desktop\Leiden University CS - AI\Deep Learning\Assignment 1\labels.npy'
    
    
    X_train, y_train, X_val, y_val, X_test, y_test = load_and_split_data(images_path=images_path, labels_path=labels_path)

    y_test1 = label_transformation(y_test)
    y_train1 = label_transformation(y_train)
    y_val1 = label_transformation(y_val)

    width, height = X_train.shape[1], X_train.shape[2]
    input_shape = (width, height, 1)
    num_classes = y_test1.shape[1]

    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),  # Prevents overfitting

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.25),   # Further prevents overfitting
        Dense(num_classes, activation='softmax')  # Output layer with softmax activation
    ])
        
    model.compile(
        loss='categorical_crossentropy',     
        optimizer='adam',                    
        metrics=['accuracy']               
    )


    model.fit(
        X_train, y_train1,
        batch_size=128,
        epochs=20,
        verbose=1,
        validation_data=(X_val, y_val1)   
    )


    # Evaluate the model
    score = model.evaluate(X_test, y_test1, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])



    # run a model with output is 720 neurons - classification
    # run a model where output is 2 neurons - regression - only this

    # provide accuracy as  loss function i s- common sense difference between predicted and target values 
    # objective - minimize this loss function

    # 4 types of model - classification, regreesion, multi-head models, label transformation (optional)

